# CS246 - Homework 1

## Question 2

### Association Rules

### Setup

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Download the data

In [ ]:
id='1gF1gGOzCJwJnduOl4ID07jYhlyqh5lyh'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('browsing.txt')

Import library

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.mllib.fpm import FPGrowth
import pandas as pd
import itertools
from functools import partial
from itertools import combinations

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
lines=sc.textFile('/content/browsing.txt')

In [ ]:
#sử dụng PySpark để tính toán tập các mục (itemsets) phổ biến
frequent_items=lines.flatMap(lambda l:l.split()).map(lambda ele:(ele,1)).\
  reduceByKey(lambda e1,e2:e1+e2).filter(lambda x:x[1]>=100)

#chuyển đổi kết quả tính toán từ đối tượng RDD của PySpark sang một từ điển Python thông thường.
frequent_itemset=frequent_items.collectAsMap()

#sử dụng PySpark để tính toán tập các cặp (pairs) phổ biến từ một tập dữ liệu (được lưu trữ trong biến lines) và từ tập frequent_itemset chứa các mục phổ biến.
frequent_pairs = lines.map(lambda l: l.split()).flatMap(partial(combinations, r=2)).map(lambda pair: sorted(pair)) \
  .map(lambda pair: (tuple(pair), 1)).filter(
  lambda ele: ele[0][0] in frequent_itemset and ele[0][1] in frequent_itemset).reduceByKey(
  lambda p1, p2: p1 + p2).filter(lambda x: x[1] >= 100)

##chuyển đổi kết quả tính toán từ đối tượng RDD của PySpark sang một từ điển Python thông thường.
freq_pairs_count=frequent_pairs.collectAsMap()

#chuyển đổi tập các cặp (pairs) phổ biến (được lưu trữ trong biến frequent_pairs) sang một định dạng mới, 
#trong đó mỗi cặp được biểu diễn bằng hai cặp khác, một cặp với thứ tự ban đầu và một cặp với thứ tự đảo ngược.
frequent_pairs=frequent_pairs.flatMap(lambda ele:[((ele[0][0],ele[0][1]),ele[1]),((ele[0][1],ele[0][0]),ele[1])])
frequent_pair_conf=frequent_pairs.map(lambda ele:(ele[0],float(ele[1]/frequent_itemset[ele[0][0]]))).sortBy(lambda x:-x[1])

In [ ]:
#tạo ra tất cả các tập ba (triplets) từ tập dữ liệu 
triples=lines.map(lambda l:l.split()).flatMap(partial(combinations,r=3)).map(lambda triple:tuple(list(sorted(triple))))

#tính toán tập các tập ba (triplets) phổ biến
triples=triples.map(lambda triple:(triple,1)).reduceByKey(lambda t1,t2:t1+t2).filter(lambda x:x[1]>=100)

#tính toán các quy tắc kết hợp (association rules) phổ biến và 
#độ tin cậy tương ứng của chúng từ tập các tập ba (triplets) phổ biến (được lưu trữ trong biến triples) 
#và tần số xuất hiện của các cặp phổ biến (được lưu trữ trong biến freq_pairs_count).
freq_triples_conf=triples.flatMap(lambda ele:[(((ele[0][0],ele[0][1]),ele[0][2]),ele[1]),
                                              (((ele[0][0],ele[0][2]),ele[0][1]),ele[1]),
                                              (((ele[0][1],ele[0][2]),ele[0][0]),ele[1])])\
.reduceByKey(lambda t1,t2:t1+t2).map(lambda ele:(ele[0],ele[1]/freq_pairs_count[ele[0][0]])).sortBy(lambda x:(-x[1],x[0][0],x[0][1]))\
                    .map(lambda ele:(ele[0][0],ele[0][1],ele[1]))

In [ ]:
frequent_pair_conf.toDF().show(5)

+--------------------+------------------+
|                  _1|                _2|
+--------------------+------------------+
|{DAI93865, FRO40251}|               1.0|
|{GRO85051, FRO40251}| 0.999176276771005|
|{GRO38636, FRO40251}|0.9906542056074766|
|{ELE12951, FRO40251}|0.9905660377358491|
|{DAI88079, FRO40251}|0.9867256637168141|
+--------------------+------------------+
only showing top 5 rows



In [ ]:
freq_triples_conf.toDF().show(5)

+--------------------+--------+---+
|                  _1|      _2| _3|
+--------------------+--------+---+
|{DAI23334, ELE92920}|DAI62779|1.0|
|{DAI31081, GRO85051}|FRO40251|1.0|
|{DAI55911, GRO85051}|FRO40251|1.0|
|{DAI62779, DAI88079}|FRO40251|1.0|
|{DAI75645, GRO85051}|FRO40251|1.0|
+--------------------+--------+---+
only showing top 5 rows

